In [1]:
import pandas as pd

In [19]:
# training set
access_path1 = 'batch1/batch1_rest_result/voted_result/Batch1_rest_access_voted_result.xlsx'
access_path2 = 'batch2/batch2_result/voted_result/Batch2_access_voted_result.xlsx'
costs_path = 'batch1/batch1_rest_result/voted_result/Batch1_rest_costs_voted_result.xlsx'
errors_path = 'batch1/batch1_rest_result/voted_result/Batch1_rest_errors_voted_result.xlsx'
delays_path = 'batch1/batch1_rest_result/voted_result/Batch1_rest_delays_voted_result.xlsx'
trusts_path1 = 'batch1/batch1_rest_result/voted_result/Batch1_rest1_trusts_voted_result.xlsx'
trusts_path2 = 'batch1/batch1_rest_result/voted_result/Batch1_rest2_trusts_voted_result.xlsx'

In [20]:
access1_df = pd.read_excel(access_path1)
access2_df = pd.read_excel(access_path2)
access_df = pd.concat([access1_df, access2_df])
trusts1_df = pd.read_excel(trusts_path1)
trusts2_df = pd.read_excel(trusts_path2)
trusts_df = pd.concat([trusts1_df, trusts2_df])
costs_df = pd.read_excel(costs_path)
errors_df = pd.read_excel(errors_path)
delays_df = pd.read_excel(delays_path)
df_dic = {'access': access_df, 'costs': costs_df, 'errors':errors_df, 'delays': delays_df, 'trusts': trusts_df}

In [2]:
# test set
access_path = 'batch1/batch1_1000_result/voted_result/Batch_1000_access_voted_result.xlsx'
costs_path = 'batch1/batch1_1000_result/voted_result/Batch_1000_costs_voted_result.xlsx'
errors_path = 'batch1/batch1_1000_result/voted_result/Batch_1000_errors_voted_result.xlsx'
delays_path = 'batch1/batch1_1000_result/voted_result/Batch_1000_delays_voted_result.xlsx'
trusts_path = 'batch1/batch1_1000_result/voted_result/Batch_1000_trusts_voted_result.xlsx'

In [4]:
access_df = pd.read_excel(access_path)
costs_df = pd.read_excel(costs_path)
errors_df = pd.read_excel(errors_path)
delays_df = pd.read_excel(delays_path)
trusts_df = pd.read_excel(trusts_path )
df_dic = {'access': access_df, 'costs': costs_df, 'errors':errors_df, 'delays': delays_df, 'trusts': trusts_df}

In [5]:
def replace(x, name):
    ans = []
    for each in x[1:-1].split(','):
        if '0' not in each:
            ans.append(name + '-' + each.strip())
#             ans.append(name + '-' + each.strip()[1:-1])
        else:
            ans.append('not about')
    return ans
for name, df in df_dic.items():
    df['ground_truth_subaspect'] = df['ground_truth_subaspect'].apply(lambda x: replace(x, name))

In [6]:
frames = [access_df, costs_df, errors_df, delays_df, trusts_df]
concat_df = pd.concat(frames)
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5001 entries, 0 to 999
Data columns (total 11 columns):
aspect                    5001 non-null object
course_of_problem         5001 non-null object
group                     5001 non-null object
id                        5001 non-null object
test                      5001 non-null object
text                      5001 non-null object
trainOrtest               5001 non-null object
treatment                 5001 non-null object
batch                     5001 non-null int64
ground_truth_subaspect    5001 non-null object
ground_truth_sentiment    5001 non-null int64
dtypes: int64(2), object(9)
memory usage: 468.8+ KB


In [7]:
concat_df.head()

,aspect,course_of_problem,group,id,test,text,trainOrtest,treatment,batch,ground_truth_subaspect,ground_truth_sentiment
0,[3],[],Menopause,Menopause-388293-4-[6],[],bloody hard ! And surely you do not expect any...,train,[],1,[access-3],0
1,[3],[],Sexual_Health,Sexual_Health-498510-0-14,['all of those tests'],pdf ? dl0 I did all of those tests after own r...,train,[],1,[access-2],0
2,[3],[],Shingles,Shingles--20440-25-3,[],This morning I woke up with my second bout of ...,train,[],1,[not about],1
3,[3],[],Prostate_Problems,Prostate_Problems-301859-477-9,[],I hope that the FDA will approve soon as this ...,train,[],1,[not about],0
4,[3],[],Benzodiazepines_and_Z_Drugs,Benzodiazepines_and_Z_Drugs-502393-12-2[6],"['every test', 'my heart and brain']","in hand , I had every test on my heart and bra...",train,[],1,[not about],1


In [8]:
from collections import Counter
def merge_aspect(series):
    ans = set()
    for ground_truth in series:
        for option in ground_truth:
            ans.add(option)
    if len(ans) > 1 and 'not about' in ans:
        ans.remove('not about')
    return list(ans)
def merge_sentiment(series):
    counter = Counter()
    for each in series:
        counter[each] += 1
    return counter.most_common(1)[0][0]

In [9]:
merged_subaspect_df = concat_df.groupby(['id']).agg({'ground_truth_subaspect':merge_aspect}).reset_index()
merged_sentiment_df = concat_df.groupby(['id']).agg({'ground_truth_sentiment':merge_sentiment}).reset_index()
merged_ground_truth_df = pd.merge(merged_subaspect_df, merged_sentiment_df, how='inner', on='id')
unique_df = concat_df.drop(['ground_truth_subaspect', 'ground_truth_sentiment'], axis=1).drop_duplicates()
complete_df = pd.merge(unique_df, merged_ground_truth_df, how='inner', on='id')

In [10]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 11 columns):
aspect                    5000 non-null object
course_of_problem         5000 non-null object
group                     5000 non-null object
id                        5000 non-null object
test                      5000 non-null object
text                      5000 non-null object
trainOrtest               5000 non-null object
treatment                 5000 non-null object
batch                     5000 non-null int64
ground_truth_subaspect    5000 non-null object
ground_truth_sentiment    5000 non-null int64
dtypes: int64(2), object(9)
memory usage: 468.8+ KB


In [11]:
complete_df.head()

,aspect,course_of_problem,group,id,test,text,trainOrtest,treatment,batch,ground_truth_subaspect,ground_truth_sentiment
0,[3],[],Menopause,Menopause-388293-4-[6],[],bloody hard ! And surely you do not expect any...,train,[],1,[access-3],0
1,[3],[],Sexual_Health,Sexual_Health-498510-0-14,['all of those tests'],pdf ? dl0 I did all of those tests after own r...,train,[],1,[access-2],0
2,[3],[],Shingles,Shingles--20440-25-3,[],This morning I woke up with my second bout of ...,train,[],1,[not about],1
3,[3],[],Prostate_Problems,Prostate_Problems-301859-477-9,[],I hope that the FDA will approve soon as this ...,train,[],1,[not about],0
4,[3],[],Benzodiazepines_and_Z_Drugs,Benzodiazepines_and_Z_Drugs-502393-12-2[6],"['every test', 'my heart and brain']","in hand , I had every test on my heart and bra...",train,[],1,[not about],1


In [12]:
complete_df['ground_truth_subaspect'].value_counts()/(complete_df.shape[0]) 

[not about]                       0.7910
[access-2]                        0.0430
[costs-2]                         0.0270
[access-1]                        0.0180
[errors-1]                        0.0178
[trusts-1]                        0.0142
[delays-2]                        0.0140
[trusts-2]                        0.0132
[delays-1]                        0.0112
[delays-3]                        0.0108
[costs-1]                         0.0092
[access-3]                        0.0084
[costs-4]                         0.0068
[costs-4, costs-2]                0.0036
[errors-2]                        0.0022
[trusts-4]                        0.0012
[costs-4, costs-1]                0.0012
[costs-1, costs-2]                0.0012
[errors-3]                        0.0012
[access-4]                        0.0010
[access-2, access-3]              0.0006
[trusts-3]                        0.0006
[costs-4, costs-1, costs-2]       0.0004
[trusts-2, trusts-4]              0.0004
[trusts-1, trust

In [13]:
complete_df = complete_df.sample(frac=1, random_state=1)
complete_df['text'] = complete_df['text'].apply(lambda x: x.lower())

In [14]:
complete_df.to_excel('medical_sieve_test_set_lower.xlsx', index=False)

In [15]:
with open('medical_sieve_test_set_text_lower.txt', 'w') as f:
    for each_sentence in complete_df['text']:
        f.write(each_sentence + '\n')